## Lightfoot Tech assessment. 

Data exploration is described in this Notebook to show my working. 
The final data cleaning, processing, and creation of the SQL database was run using the python scripts (src/main.py)

##### Step 1: 
Explore the data provided in python

In [203]:
import pandas as pd 
import numpy as np
import glob

#What files do we have? 
path = "/Users/alexjvr/Dropbox/Tech_Assessment"
all_files = glob.glob(path + "/*.csv")
print(all_files)


#Read vehicle data
vehicle_df = pd.read_csv(all_files[1], na_values=(np.NaN))
vehicle_df.info()

#Read purchase data
purchase_df = pd.read_csv(all_files[0], na_values=(np.NaN))
purchase_df.info()


['/Users/alexjvr/Dropbox/Tech_Assessment/lf_tech_test_purchase_data.csv', '/Users/alexjvr/Dropbox/Tech_Assessment/lf_tech_test_vehicle_data.csv']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978 entries, 0 to 977
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   vehicle_id      978 non-null    int64  
 1   name            978 non-null    object 
 2   description     922 non-null    object 
 3   make            978 non-null    object 
 4   model           978 non-null    object 
 5   type            978 non-null    object 
 6   year            978 non-null    int64  
 7   price           978 non-null    float64
 8   engine          976 non-null    object 
 9   cylinders       876 non-null    float64
 10  fuel            971 non-null    object 
 11  mileage         944 non-null    float64
 12  transmission    976 non-null    object 
 13  trim            977 non-null    object 
 14  body            975 non-

Dataframes have vehicle_id column in common, so I can create an SQL database. 
This will also make future queries easier. 
Clean the data first. 

##### Step 2a: 

Clean vehicle data

In [208]:
import datetime as dt


# Look for missing data. Obvious text errors. And numerical columns that are in incorrect format, 
# unexpected ranges for numbers, and outlier values. 
# Expected dataset has 978 rows
vehicle_df.head()
vehicle_df.tail()


vehicle_df.describe(include='all')

# Anything obviously wrong with entries with missing data?
vehicle_df[vehicle_df.isnull().any(axis=1)]

# Anything particularly strange about entries with multiple Nan values? 
vehicle_df.loc[vehicle_df.isnull().sum(1)>1]

# Several of these entries have [engine:'c']. There are variations of this engine type 
# ("c Motor", and "c ZEV"), so I will leave this in for now, but it needs to be checked. 
# Replace erroneous engine types "<dt>VIN</dt>\"* with Nan
vehicle_df["engine"].replace('.*VIN*', np.NaN, regex=True, inplace=True)


#Cars cannot have 0 cylinders, so mark as missing data
vehicle_df["cylinders"] = np.where(vehicle_df["cylinders"]<1, np.NaN, vehicle_df["cylinders"])


#cars with 0 mileage? I'm not sure if cars can have 0 mileage. This will need to be checked with someone. 
vehicle_df[vehicle_df.mileage < 10]
#car with >100 mileage? All looks within the range of possibility
vehicle_df[vehicle_df.mileage > 100]

#price: looks within the range of possibility 
vehicle_df.price.describe()

# year is within a sensible range (2023-2025)
vehicle_df.year.describe()

# Fuel: All words are unique and are types of fuel
vehicle_df.fuel.unique()

#Range of vehicle_id (to compare with purchase_df)
vehicle_df.vehicle_id.describe()
vehicle_df.vehicle_id.nunique()
vehicle_uniq_vehicle_id = vehicle_df.vehicle_id.unique()




/var/folders/d1/2957hj5928x_7dc6xjx_2rx40000gq/T/ipykernel_10875/2358857759.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  vehicle_df["engine"].replace('.*VIN*', np.NaN, regex=True, inplace=True)


##### Step 2b: 

Clean purchase data

In [209]:
purchase_df.head()
purchase_df.tail()
purchase_df.describe()

# Anything obviously wrong with entries with missing data? There's no missing data!
purchase_df[purchase_df.isnull().any(axis=1)]


#All customer_id unique? Yes
purchase_df.customer_id.nunique()

#All customer_uuid entries unique? Yes
purchase_df.customer_uuid.nunique()
#Quick check for any unexpected entries in the customer_uuid
purchase_df[~purchase_df["customer_uuid"].str.contains("-")]
purchase_df["customer_uuid"].str.findall(r'[A-Z]')


# Split purchase date into year, month, day
purchase_df.purchase_date = pd.to_datetime(purchase_df["purchase_date"])
purchase_df['purchase_day'] = purchase_df['purchase_date'].dt.day
purchase_df['purchase_month'] = purchase_df['purchase_date'].dt.month
purchase_df['purchase_year'] = purchase_df['purchase_date'].dt.year
purchase_df.head()


#rrp_discount: correct format and within sensible range? Yes. Range 0-0.18
purchase_df.rrp_discount.describe()
purchase_df.rrp_discount.unique()

#vehicle_id within sensible range? Range looks good
purchase_df.vehicle_id.describe()
purchase_df.vehicle_id.nunique()

#Do we have the same set of uniqe vehicle_ids in both dataframes? Yes. So we can use this as the key column 
purchase_uniq_vehicle_id = purchase_df.vehicle_id.unique()
list(set(purchase_uniq_vehicle_id)-set(vehicle_uniq_vehicle_id))


#Summary of city names. Anything weird? No. All are existing cities and there are no variations in spelling
purchase_df.city.unique()
purchase_df.city.nunique()

#summary of approx_population. Within sensible range, and the same number of unique values as we have cities. 
purchase_df.approx_population.describe()
purchase_df.approx_population.nunique()

#Format of all columns
purchase_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 12 columns):
 #   Column             Dtype         
---  ------             -----         
 0   customer_id        int64         
 1   customer_uuid      object        
 2   purchase_date      datetime64[ns]
 3   rrp_discount       float64       
 4   vehicle_id         int64         
 5   city               object        
 6   lat                float64       
 7   lng                float64       
 8   approx_population  int64         
 9   purchase_day       int32         
 10  purchase_month     int32         
 11  purchase_year      int32         
dtypes: datetime64[ns](1), float64(3), int32(3), int64(3), object(2)
memory usage: 160.2+ MB


978